In [ ]:
import sqlite3
import logging
from typing import Optional, List, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class AncestryDatabase:
    """
    A class to interact with an SQLite3 database for ancestry data.
    """

    def __init__(self, db_name: str = "ancestry.db"):
        self.db_name = db_name
        self.conn = None
        self.cursor = None

    def connect(self) -> None:
        """Establishes a connection to the database."""
        try:
            self.conn = sqlite3.connect(self.db_name)
            self.cursor = self.conn.cursor()
            logging.info(f"Connected to database: {self.db_name}")
        except sqlite3.Error as e:
            logging.error(f"Database connection error: {e}")

    def close(self) -> None:
        """Closes the database connection."""
        if self.conn:
            self.conn.commit()
            self.conn.close()
            logging.info("Database connection closed.")

    def create_tables(self) -> None:
        """Creates necessary tables if they do not exist."""
        try:
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS People (
                    person_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    birth_date TEXT,
                    death_date TEXT
                )
            ''')
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS Relationships (
                    relationship_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    person1_id INTEGER,
                    person2_id INTEGER,
                    relationship_type TEXT,
                    UNIQUE(person1_id, person2_id, relationship_type),
                    FOREIGN KEY(person1_id) REFERENCES People(person_id) ON DELETE CASCADE,
                    FOREIGN KEY(person2_id) REFERENCES People(person_id) ON DELETE CASCADE
                )
            ''')
            logging.info("Tables created successfully.")
        except sqlite3.Error as e:
            logging.error(f"Error creating tables: {e}")

    def add_person(self, first_name: str, last_name: str, birth_date: Optional[str] = None, death_date: Optional[str] = None) -> Optional[int]:
        """Adds a person to the People table and returns their ID."""
        try:
            self.cursor.execute('''
                INSERT INTO People (first_name, last_name, birth_date, death_date) 
                VALUES (?, ?, ?, ?)
            ''', (first_name, last_name, birth_date, death_date))
            self.conn.commit()
            logging.info(f"Person '{first_name} {last_name}' added successfully.")
            return self.cursor.lastrowid
        except sqlite3.Error as e:
            logging.error(f"Error adding person: {e}")
            return None

    def add_relationship(self, person1_id: int, person2_id: int, relationship_type: str) -> None:
        """Adds a relationship between two people."""
        try:
            self.cursor.execute('''
                INSERT INTO Relationships (person1_id, person2_id, relationship_type) 
                VALUES (?, ?, ?)
            ''', (person1_id, person2_id, relationship_type))
            self.conn.commit()
            logging.info(f"Relationship added successfully.")
        except sqlite3.IntegrityError:
            logging.warning("Duplicate relationship detected.")
        except sqlite3.Error as e:
            logging.error(f"Error adding relationship: {e}")
